<a href="https://colab.research.google.com/github/talkin24/jaxflax_lab/blob/main/Model_inspection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Model surgery

일반적으로 Flax 모듈과 옵티마이저는 파라미터를 추적하고 업데이트합니다. 하지만 모델 작업을 수행하고 파라미터 텐서를 직접 조정하고 싶을 때가 있을 수 있습니다. 이 가이드는 그 방법을 보여줍니다.

## Setup

In [1]:
!pip install --upgrade -q pip jax jaxlib flax


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 34.7 MB/s eta 0:00:00


In [2]:
import functools

import jax
import jax.numpy as jnp
from flax import traverse_util
from flax import linen as nn
from flax.core import freeze
import jax
import optax

## Surgery with Flax Modules

데모를 위해 작은 컨볼루션 신경망 모델을 만들어 보겠습니다.

평소처럼 `CNN.init(...)['params']`를 실행하여 훈련의 모든 단계에서 `params`를 전달하고 수정할 수 있습니다.

In [3]:
class CNN(nn.Module):
    @nn.compact
    def __call__(self, x):
      x = nn.Conv(features=32, kernel_size=(3, 3))(x)
      x = nn.relu(x)
      x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
      x = nn.Conv(features=64, kernel_size=(3, 3))(x)
      x = nn.relu(x)
      x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
      x = x.reshape((x.shape[0], -1))
      x = nn.Dense(features=256)(x)
      x = nn.relu(x)
      x = nn.Dense(features=10)(x)
      x = nn.log_softmax(x)
      return x

def get_initial_params(key):
    init_shape = jnp.ones((1, 28, 28, 1), jnp.float32)
    initial_params = CNN().init(key, init_shape)['params']
    return initial_params

key = jax.random.PRNGKey(0)
params = get_initial_params(key)

jax.tree_util.tree_map(jnp.shape, params)

FrozenDict({
    Conv_0: {
        bias: (32,),
        kernel: (3, 3, 1, 32),
    },
    Conv_1: {
        bias: (64,),
        kernel: (3, 3, 32, 64),
    },
    Dense_0: {
        bias: (256,),
        kernel: (3136, 256),
    },
    Dense_1: {
        bias: (10,),
        kernel: (256, 10),
    },
})

`params`로 반환되는 것은 커널과 바이어스로 몇 개의 JAX 배열을 포함하는 `FrozenDict`입니다.

`FrozenDict`는 읽기 전용 딕셔너리에 불과하며, Flax는 JAX 배열이 불변이고 새 `params`가 이전 `params`를 대체해야 하는 JAX의 기능적 특성 때문에 이를 읽기 전용으로 만들었습니다. 딕셔너리를 읽기 전용으로 만들면 학습 및 업데이트 중에 실수로 딕셔너리가 제자리에서 변경되지 않도록 할 수 있습니다.

Flax 모듈 외부에서 실제로 매개변수를 수정하는 한 가지 방법은 명시적으로 플랫화하여 변경 가능한 딕셔너리를 만드는 것입니다. 중첩된 모든 키를 결합하려면 구분 기호 `sep`을 사용할 수 있습니다. `sep`을 지정하지 않으면 키는 중첩된 모든 키의 튜플이 됩니다.

In [7]:
# Get a flattened key-value list.
flat_params = traverse_util.flatten_dict(params, sep='/')

jax.tree_util.tree_map(jnp.shape, flat_params)

{'Conv_0/bias': (32,),
 'Conv_0/kernel': (3, 3, 1, 32),
 'Conv_1/bias': (64,),
 'Conv_1/kernel': (3, 3, 32, 64),
 'Dense_0/bias': (256,),
 'Dense_0/kernel': (3136, 256),
 'Dense_1/bias': (10,),
 'Dense_1/kernel': (256, 10)}

이제 파라미터를 원하는 대로 설정할 수 있습니다. 작업이 끝나면 다시 평평하게 만들어 향후 훈련에 사용하세요.

In [8]:
# Somehow modify a layer
dense_kernel = flat_params['Dense_1/kernel']
flat_params['Dense_1/kernel'] = dense_kernel / jnp.linalg.norm(dense_kernel)

# Unflatten.
unflat_params = traverse_util.unflatten_dict(flat_params, sep='/')
# Refreeze.
unflat_params = freeze(unflat_params)
jax.tree_util.tree_map(jnp.shape, unflat_params)

FrozenDict({
    Conv_0: {
        bias: (32,),
        kernel: (3, 3, 1, 32),
    },
    Conv_1: {
        bias: (64,),
        kernel: (3, 3, 32, 64),
    },
    Dense_0: {
        bias: (256,),
        kernel: (3136, 256),
    },
    Dense_1: {
        bias: (10,),
        kernel: (256, 10),
    },
})

## Surgery with Optimizers

`Optax`를 옵티마이저로 사용할 때 `opt_state`는 실제로 옵티마이저를 구성하는 개별 그라데이션 변환의 상태가 중첩된 튜플입니다. 

이러한 상태에는 매개변수 트리를 미러링하는 pytree가 포함되어 있으며, 평탄화, 수정, 평탄화 해제, 원래 상태를 미러링하는 새로운 옵티마이저 상태 재생성 등 동일한 방식으로 수정할 수 있습니다.

In [9]:
tx = optax.adam(1.0)
opt_state = tx.init(params)

# The optimizer state is a tuple of gradient transformation states.
jax.tree_util.tree_map(jnp.shape, opt_state)

(ScaleByAdamState(count=(), mu=FrozenDict({
     Conv_0: {
         bias: (32,),
         kernel: (3, 3, 1, 32),
     },
     Conv_1: {
         bias: (64,),
         kernel: (3, 3, 32, 64),
     },
     Dense_0: {
         bias: (256,),
         kernel: (3136, 256),
     },
     Dense_1: {
         bias: (10,),
         kernel: (256, 10),
     },
 }), nu=FrozenDict({
     Conv_0: {
         bias: (32,),
         kernel: (3, 3, 1, 32),
     },
     Conv_1: {
         bias: (64,),
         kernel: (3, 3, 32, 64),
     },
     Dense_0: {
         bias: (256,),
         kernel: (3136, 256),
     },
     Dense_1: {
         bias: (10,),
         kernel: (256, 10),
     },
 })),
 EmptyState())

옵티마이저 상태 내부의 파이트리는 파라미터와 동일한 구조를 따르며 정확히 동일한 방식으로 평평하게 하거나 수정할 수 있습니다.

In [10]:
flat_mu = traverse_util.flatten_dict(opt_state[0].mu, sep='/')
flat_nu = traverse_util.flatten_dict(opt_state[0].nu, sep='/')

jax.tree_util.tree_map(jnp.shape, flat_mu)

{'Conv_0/bias': (32,),
 'Conv_0/kernel': (3, 3, 1, 32),
 'Conv_1/bias': (64,),
 'Conv_1/kernel': (3, 3, 32, 64),
 'Dense_0/bias': (256,),
 'Dense_0/kernel': (3136, 256),
 'Dense_1/bias': (10,),
 'Dense_1/kernel': (256, 10)}

수정 후 최적화 상태를 다시 생성합니다. 향후 훈련에 활용하세요.

In [11]:
opt_state = (
    opt_state[0]._replace(
        mu=traverse_util.unflatten_dict(flat_mu, sep='/'),
        nu=traverse_util.unflatten_dict(flat_nu, sep='/'),
    ),
) + opt_state[1:]
jax.tree_util.tree_map(jnp.shape, opt_state)

(ScaleByAdamState(count=(), mu={'Conv_0': {'bias': (32,), 'kernel': (3, 3, 1, 32)}, 'Conv_1': {'bias': (64,), 'kernel': (3, 3, 32, 64)}, 'Dense_0': {'bias': (256,), 'kernel': (3136, 256)}, 'Dense_1': {'bias': (10,), 'kernel': (256, 10)}}, nu={'Conv_0': {'bias': (32,), 'kernel': (3, 3, 1, 32)}, 'Conv_1': {'bias': (64,), 'kernel': (3, 3, 32, 64)}, 'Dense_0': {'bias': (256,), 'kernel': (3136, 256)}, 'Dense_1': {'bias': (10,), 'kernel': (256, 10)}}),
 EmptyState())

# Extracting intermediate values

이 패턴은 모듈에서 중간 값을 추출하는 방법을 보여줍니다. `nn.compact`를 사용하는 간단한 CNN부터 시작해 보겠습니다.

In [12]:
from flax import linen as nn
import jax
import jax.numpy as jnp
from typing import Sequence

class CNN(nn.Module):
  @nn.compact
  def __call__(self, x):
    x = nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = x.reshape((x.shape[0], -1))  # flatten
    x = nn.Dense(features=256)(x)
    x = nn.relu(x)
    x = nn.Dense(features=10)(x)
    x = nn.log_softmax(x)
    return x

이 모듈은 `nn.compact`를 사용하기 때문에 중간 값에 직접 액세스할 수 없습니다. 중간값을 노출하는 몇 가지 방법이 있습니다.

## Store intermediate values in a new variable collection

CNN은 다음과 같이 중간체를 저장하기 위한 `sow` 호출로 보강할 수 있습니다:

In [13]:
# Default CNN
class CNN(nn.Module):
  @nn.compact
  def __call__(self, x):
    x = nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = x.reshape((x.shape[0], -1))  # flatten

    x = nn.Dense(features=256)(x)
    x = nn.relu(x)
    x = nn.Dense(features=10)(x)
    x = nn.log_softmax(x)
    return x

# CNN using sow API
class SowCNN(nn.Module):
  @nn.compact
  def __call__(self, x):
    x = nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = x.reshape((x.shape[0], -1))  # flatten
    self.sow('intermediates', 'features', x) #####
    x = nn.Dense(features=256)(x)
    x = nn.relu(x)
    x = nn.Dense(features=10)(x)
    x = nn.log_softmax(x)
    return x

`sow`는 변수 컬렉션이 변경 가능하지 않을 때 no-op으로 작동합니다. 따라서 디버깅 및 중간체의 선택적 추적에 완벽하게 작동합니다. 'intermediates' 컬렉션은 `capture_intermediates` API에서도 사용됩니다(마지막 섹션 참조).

기본적으로 `sow`는 호출될 때마다 값을 추가한다는 점에 유의하세요:

- 이는 모듈이 인스턴스화되면 부모 모듈에서 여러 번 호출될 수 있고, 모든 파종된 값을 포착하고 싶기 때문에 필요합니다.

- 따라서 `변수`에 중간 값을 다시 넣지 않도록 해야 합니다. 그렇지 않으면 호출할 때마다 해당 튜플의 길이가 증가하고 재컴파일이 트리거됩니다.

- 기본 추가 동작을 재정의하려면 `Module.sow()`를 참조하여 `init_fn` 및 `reduce_fn`을 지정하세요.

In [14]:
class SowCNN2(nn.Module):
  @nn.compact
  def __call__(self, x):
    mod = SowCNN(name='SowCNN')
    return mod(x) + mod(x)  # Calling same module instance twice.

@jax.jit
def init(key, x):
  variables = SowCNN2().init(key, x)
  # By default the 'intermediates' collection is not mutable during init.
  # So variables will only contain 'params' here.
  return variables

@jax.jit
def predict(variables, x):
  # If mutable='intermediates' is not specified, then .sow() acts as a noop.
  output, mod_vars = SowCNN2().apply(variables, x, mutable='intermediates')
  features = mod_vars['intermediates']['SowCNN']['features']
  return output, features

batch = jnp.ones((1,28,28,1))
variables = init(jax.random.PRNGKey(0), batch)
preds, feats = predict(variables, batch)

assert len(feats) == 2  # Tuple with two values since module was called twice.

In [15]:
feats

(Array([[0.1793044 , 0.        , 0.3081015 , ..., 0.2719369 , 0.03271667,
         0.40916097]], dtype=float32),
 Array([[0.1793044 , 0.        , 0.3081015 , ..., 0.2719369 , 0.03271667,
         0.40916097]], dtype=float32))

## Refactor module into submodules

이 패턴은 서브모듈을 분할할 특정 방식이 분명한 경우에 유용합니다. `setup`에서 노출하는 모든 서브모듈을 직접 사용할 수 있습니다. 제한적으로 `setup`에서 모든 서브모듈을 정의하고 `nn.compact`를 전혀 사용하지 않을 수 있습니다.

In [16]:
class RefactoredCNN(nn.Module):
  def setup(self):
    self.features = Features()
    self.classifier = Classifier()

  def __call__(self, x):
    x = self.features(x)
    x = self.classifier(x)
    return x

class Features(nn.Module):
  @nn.compact
  def __call__(self, x):
    x = nn.Conv(features=32, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = nn.Conv(features=64, kernel_size=(3, 3))(x)
    x = nn.relu(x)
    x = nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2))
    x = x.reshape((x.shape[0], -1))  # flatten
    return x

class Classifier(nn.Module):
  @nn.compact
  def __call__(self, x):
    x = nn.Dense(features=256)(x)
    x = nn.relu(x)
    x = nn.Dense(features=10)(x)
    x = nn.log_softmax(x)
    return x

@jax.jit
def init(key, x):
  variables = RefactoredCNN().init(key, x)
  return variables['params']

@jax.jit
def features(params, x):
  return RefactoredCNN().apply({"params": params}, x,
    method=lambda module, x: module.features(x))

params = init(jax.random.PRNGKey(0), batch)

features(params, batch)

Array([[0.        , 0.01445577, 0.73324203, ..., 0.        , 0.        ,
        0.21551636]], dtype=float32)

## Use `capture_intermediates`

Linen은 코드 변경 없이 서브모듈에서 중간 반환값을 자동으로 캡처할 수 있도록 지원합니다. 이 패턴은 중간값을 캡처하는 "sledge hammer" 접근 방식으로 간주해야 합니다. 디버깅 및 검사 도구로서 매우 유용하지만 이 하우투에 설명된 다른 패턴을 사용하는 것도 좋습니다.

다음 코드 예제에서는 중간 활성화가 비한정(NaN 또는 무한)인지 여부를 확인합니다:

In [17]:
@jax.jit
def init(key, x):
  variables = CNN().init(key, x)
  return variables

@jax.jit
def predict(variables, x):
  y, state = CNN().apply(variables, x, capture_intermediates=True, mutable=["intermediates"])
  intermediates = state['intermediates']
  fin = jax.tree_util.tree_map(lambda xs: jnp.all(jnp.isfinite(xs)), intermediates)
  return y, fin

variables = init(jax.random.PRNGKey(0), batch)
y, is_finite = predict(variables, batch)
all_finite = all(jax.tree_util.tree_leaves(is_finite))
assert all_finite, "non-finite intermediate detected!"

In [18]:
is_finite

FrozenDict({
    Conv_0: {
        __call__: (Array(True, dtype=bool),),
    },
    Conv_1: {
        __call__: (Array(True, dtype=bool),),
    },
    Dense_0: {
        __call__: (Array(True, dtype=bool),),
    },
    Dense_1: {
        __call__: (Array(True, dtype=bool),),
    },
    __call__: (Array(True, dtype=bool),),
})

기본적으로 `__call__` 메서드의 중간체만 수집됩니다. 

또는 `Module` 인스턴스와 메서드 이름을 기반으로 사용자 정의 필터를 전달할 수 있습니다.

In [19]:
filter_Dense = lambda mdl, method_name: isinstance(mdl, nn.Dense)
filter_encodings = lambda mdl, method_name: method_name == "encode"

y, state = CNN().apply(variables, batch, capture_intermediates=filter_Dense, mutable=["intermediates"])
dense_intermediates = state['intermediates']

In [20]:
dense_intermediates

FrozenDict({
    Dense_0: {
        __call__: (Array([[-0.4163544 , -0.06839216, -0.51570624, -0.19137715,  0.10282888,
                 0.5738754 ,  0.13795005,  0.18073687,  0.00480315,  0.64715713,
                 0.3356169 , -0.4829922 ,  0.4347183 ,  0.51078403, -0.449439  ,
                 0.17263898,  0.22860987,  0.08530779,  0.25815383,  0.06415588,
                -0.1518939 , -0.23635048,  0.27043548, -0.1065058 ,  0.24950364,
                 0.2170637 ,  0.11157337, -0.08237511, -0.19511226,  0.2964567 ,
                 0.02076684, -0.3986773 , -0.5505137 ,  0.6888312 , -0.3276731 ,
                 0.16291578,  0.36783996, -0.46132872, -0.32510972, -0.04895904,
                -0.36039907,  0.26155755, -0.32220736, -0.04482261, -0.20064123,
                -0.08618929,  0.55833423, -0.38625807,  0.15436846,  0.38278913,
                -0.2882816 ,  0.0843855 ,  0.4771748 , -0.14847872,  0.6558553 ,
                 0.03806313, -0.41578537,  0.01439955, -0.21037483, -0

## Use `Sequential`

`Sequential` 결합기의 간단한 구현을 사용하여 CNN을 정의할 수도 있습니다(이는 보다 상태 저장 접근 방식에서 매우 일반적입니다). 

이는 매우 간단한 모델에 유용할 수 있으며 임의의 모델 조작이 가능합니다. 하지만 매우 제한적일 수 있습니다. 조건부 하나라도 추가하려면 `Sequential`에서 벗어나 리팩터링하고 모델을 더 명시적으로 구조화해야 합니다.

In [21]:
class Sequential(nn.Module):
  layers: Sequence[nn.Module]

  def __call__(self, x):
    for layer in self.layers:
      x = layer(x)
    return x

def SeqCNN():
  return Sequential([
    nn.Conv(features=32, kernel_size=(3, 3)),
    nn.relu,
    lambda x: nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2)),
    nn.Conv(features=64, kernel_size=(3, 3)),
    nn.relu,
    lambda x: nn.avg_pool(x, window_shape=(2, 2), strides=(2, 2)),
    lambda x: x.reshape((x.shape[0], -1)),  # flatten
    nn.Dense(features=256),
    nn.relu,
    nn.Dense(features=10),
    nn.log_softmax,
  ])

@jax.jit
def init(key, x):
  variables = SeqCNN().init(key, x)
  return variables['params']

@jax.jit
def features(params, x):
  return Sequential(SeqCNN().layers[0:7]).apply({"params": params}, x)

params = init(jax.random.PRNGKey(0), batch)
features(params, batch)

Array([[0.7018405 , 0.        , 1.0706136 , ..., 0.12816615, 0.00970969,
        0.04631865]], dtype=float32)

## Extracting gradients of intermdeiate values

디버깅 목적으로 중간 값의 그라데이션을 추출하는 것이 유용할 수 있습니다. 원하는 값에 대해 `Module.perturb()` 메서드를 사용하면 이 작업을 수행할 수 있습니다.

In [22]:
class Model(nn.Module):
  @nn.compact
  def __call__(self, x):
    x = nn.relu(nn.Dense(8)(x))
    x = self.perturb('hidden', x)
    x = nn.Dense(2)(x)
    x = self.perturb('logits', x)
    return x

`perturb`는 기본적으로 `perturbations` 컬렉션에 변수를 추가하며, identity 함수처럼 동작하고 perturbation의 기울기가 입력의 기울기와 일치합니다. perturbations을 얻으려면 모델을 초기화하기만 하면 됩니다:

In [23]:
x = jnp.empty((1, 4)) # random data
y = jnp.empty((1, 2)) # random data

model = Model()
variables = model.init(jax.random.PRNGKey(1), x)
params, perturbations = variables['params'], variables['perturbations']

마지막으로 perturbation에 대한 손실의 기울기를 계산하면 중간체의 기울기와 일치하게 됩니다:

In [24]:
def loss_fn(params, perturbations, x, y):
  y_pred = model.apply({'params': params, 'perturbations': perturbations}, x)
  return jnp.mean((y_pred - y) ** 2)

intermediate_grads = jax.grad(loss_fn, argnums=1)(params, perturbations, x, y)

In [25]:
intermediate_grads

FrozenDict({
    hidden: Array([[0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32),
    logits: Array([[0., 0.]], dtype=float32),
})